In [ ]:
import pandas as pd
pd.options.display.max_rows = 5
import os
import numpy as np

In [ ]:
data_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'data'))
df = pd.read_csv(os.path.join(data_path, 'sofifa_players_201314to201718_Jan2019.csv'))

In [ ]:
df.head(2)

In [ ]:
df.columns = ['id', 'date', 'name1', 'name2', 'country', 'age', 'pos1', 'pos2', 'pos3', 'overall_rating', 'potential', 'value', 'wage', 'special_total', 'team', 'contract']

##### Get counts of types of players (based on positions/roles) in the raw data set
Also save as an Excel file for LaTeX conversion

In [ ]:
print(df.shape)
df_roles_table =  df.drop_duplicates(['name2', 'pos1', 'country'])
print(df_roles_table.shape)
df_roles_table = df.groupby(['pos1']).agg('count')
df_roles_table = df_roles_table.reset_index()
df_roles_table = df_roles_table[['pos1', 'id']]
df_roles_table.columns = ['role', 'count']
df_roles_table.sort_values(['role'])
df_roles_table.to_excel('df_roles_table.xlsx')
df_roles_table

In [ ]:
# counts of countries and clubs
print(len(df['country'].unique()))
print(len(df['team'].unique()))

In [ ]:
df = df[['date', 'name2', 'team', 'country', 'age', 'pos1', 'overall_rating', 'potential', 'value', 'wage']]

In [ ]:
df.date = df.date.str[-14:]
# df.value = df.value.str.replace('M', '')
df.value = df.value.str.replace('€', '')
# df.wage = df.wage.str.replace('K', '')
df.wage = df.wage.str.replace('€', '')
df.value = (df.value.replace(r'[KM]+$', '', regex=True).astype(float) * df.value.str.extract(r'[\d\.]+([KM]+)', expand=False) .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
df.wage = (df.wage.replace(r'[KM]+$', '', regex=True).astype(float) * df.wage.str.extract(r'[\d\.]+([KM]+)', expand=False) .fillna(1).replace(['K','M'], [10**3, 10**6]).astype(int))
# df

In [ ]:
# df = df.replace('0', np.nan).ffill() # to replace NAs with 0. Not using this method
df = df.dropna() # this may be better than replacing NAs with 0
# df

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.sort_values(by = 'date')
df.index = range(len(df.index))
# df

In [ ]:
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
df.value = pd.to_numeric(df.value)
df.wage = pd.to_numeric(df.wage)

In [ ]:
df.head(2)

In [ ]:
df['date'].unique()

In [ ]:
# separate data sets for "train" and "test" (not ML model though)
df_train = df[df['date']<pd.to_datetime('2016-05-05')]
df_test = df[df['date']>=pd.to_datetime('2016-05-05')]

In [ ]:
df_train['date'].unique()

In [ ]:
df_test['date'].unique()

In [ ]:
df_names = df.name2.unique()
len(df_names)

In [ ]:
df_train_names = df_train.name2.unique()
len(df_train_names)

In [ ]:
df_test_names = df_test.name2.unique()
len(df_test_names)

In [ ]:
test_dates = df_test['date'].unique()

In [ ]:
# function to give projections of market value, salary, rating, and potential rating (for each player)
def futureInput(df_train_player):
    n_rows = df_train_player.shape[0]
    df_train_player_future = df_train_player.tail(1)
    for i in range(len(test_dates)-1):
        df_train_player_future= df_train_player_future.append(df_train_player.tail(1))
    df_train_player_future['team'] = df_train_player['team'][n_rows-1]
    df_train_player_future['country'] = df_train_player['country'][n_rows-1]
    df_train_player_future['date'] = test_dates
    df_train_player_future['month'] = df_train_player_future['date'].dt.month
    df_train_player_future['year'] = df_train_player_future['date'].dt.year
    df_train_player_future.index = list(range(0, df_train_player_future.shape[0]))
    df_train_player_temp = df_train_player
    if df_train_player.age[n_rows-1] != df_train_player.age[n_rows-2]:
        df_train_player_future.age[:3] = df_train_player_temp.age[n_rows-1]
        df_train_player_future.age[3:7] = df_train_player_temp.age[n_rows-1] + 1
        df_train_player_future.age[7:] = df_train_player_temp.age[n_rows-1] + 2
    elif df_train_player.age[n_rows-1] != df_train_player.age[n_rows-3]:
        df_train_player_future.age[:2] = df_train_player_temp.age[n_rows-1]
        df_train_player_future.age[2:6] = df_train_player_temp.age[n_rows-1] + 1
        df_train_player_future.age[6:10] = df_train_player_temp.age[n_rows-1] + 2
        df_train_player_future.age[10:] = df_train_player_temp.age[n_rows-1] + 3
    elif df_train_player.age[n_rows-1] != df_train_player.age[n_rows-4]:
        df_train_player_future.age[:1] = df_train_player_temp.age[n_rows-1]
        df_train_player_future.age[1:5] = df_train_player_temp.age[n_rows-1] + 1
        df_train_player_future.age[5:9] = df_train_player_temp.age[n_rows-1] + 2
        df_train_player_future.age[9:] = df_train_player_temp.age[n_rows-1] + 3
    else:
        df_train_player_future.age[:4] = df_train_player_temp.age[n_rows-1] + 1
        df_train_player_future.age[4:7] = df_train_player_temp.age[n_rows-1] + 2
        df_train_player_future.age[7:] = df_train_player_temp.age[n_rows-1] + 3
    df_train_player_future = df_train_player.append(df_train_player_future)
    df_train_player_future.index = list(range(df_train_player_future.shape[0]))
    for i in range(len(test_dates)):
        df_train_player_future.overall_rating[n_rows + i] = df_train_player_future.overall_rating[n_rows - 4 + i: n_rows + i].mean()
        df_train_player_future.potential[n_rows + i] = df_train_player_future.potential[n_rows - 4 + i: n_rows + i].mean()
        df_train_player_future.value[n_rows + i] = df_train_player_future.value[n_rows - 4 + i: n_rows + i].mean()
        df_train_player_future.wage[n_rows + i] = df_train_player_future.wage[n_rows - 4 + i: n_rows + i].mean()
    return df_train_player_future

In [ ]:
# preparation for creating output data sets in the required format
n_w = 6 # n_w is the number of transfer windows in test set (including the two dummy ones - initial and final)

age_cols = ['age_'+str(s) for s in range(n_w)] 
overall_rating_cols = ['rating_'+str(s) for s in range(n_w)] 
potential_cols = ['potential_'+str(s) for s in range(n_w)] 
country_cols = ['hg_'+str(s) for s in range(n_w)] 
value_cols = ['v_'+str(s) for s in range(n_w)] 
wage_cols = ['s_'+str(s) for s in range(n_w)]

# key names for which loops are implemented below to create output val dictionaries
op_dict_keys = [age_cols, overall_rating_cols, potential_cols, country_cols, value_cols, wage_cols]

# col names for which loops are implemented below to create output val dictionaries
op_dict_cols = ['age', 'overall_rating', 'potential', 'country', 'value', 'wage']

# create empty lists for each of the above cols, and also for playerRoles, z, and y
op_dict_lists = [[] for col in op_dict_cols]
playerRoles = []
z = []
y = []

In [ ]:
counter_n = 0 
counter_p = 0
for name in df_train_names:
    counter_n = counter_n + 1
    df_train_player = df_train[df_train.name2 == name]
    df_train_player = df_train_player[df_train_player.value != 0]
    n_rows = df_train_player.shape[0]
    df_train_player.index = list(range(0, n_rows))
    if n_rows >= 4 and df_train_player.age[n_rows-1] > df_train_player.age[0]:
        df_train_temp = df_train_player
        if df_train_player.year[n_rows-1] == 2016 and df_train_player.month[n_rows-1] == 2:
            if df_train_player.year[n_rows-1] >= df_train_player.year[0]:
                counter_p = counter_p + 1
                print('Counter_p = ' + str(counter_p) + '; ' + 'Counter_n = ' + str(counter_n))
                print(df_train_player.shape)
                df_train_player_future = futureInput(df_train_player)
                # in the below line, "n_rows-1" and "n_rows+8" are the dummies, initial and final respectively. "n_rows_8" values will be set to 0
                df_train_player_temp = df_train_player_future.iloc[[n_rows-1, n_rows, n_rows+2, n_rows+4, n_rows+6, n_rows+8], :]
                df_train_player_temp.index = list(range(df_train_player_temp.shape[0]))
                playerRoles.append({'_player': df_train_player_temp.name2[0], 'pos1': df_train_player_temp.pos1[0]})
                for j, col in enumerate(op_dict_cols):
                    player_dict = {'_player': df_train_player_temp.name2[0]}
                    temp_list = df_train_player_temp[col][:n_w]
                    if col!='country':
                        temp_list[len(temp_list)-1] = 0 # set "n_rows_8" values to 0, as this corresponds to the dummy final transfer window
                    temp_dict = dict((key, value) for (key, value) in zip(op_dict_keys[j], temp_list))
                    player_dict.update(temp_dict)
                    op_dict_lists[j].append(player_dict)
                y.append({'_player': df_train_player_temp.name2[0]})
                z.append({'_player': df_train_player_temp.name2[0], 
                            'team_0': df_train_player_temp.team[0]})
playerRoles = pd.DataFrame(playerRoles)

op_dict_cols = ['age', 'overall_rating', 'potential', 'country', 'value', 'wage', 'y']
age = pd.DataFrame(op_dict_lists[0])
rating = pd.DataFrame(op_dict_lists[1])
potential = pd.DataFrame(op_dict_lists[2])
hg = pd.DataFrame(op_dict_lists[3])
v = pd.DataFrame(op_dict_lists[4])
s = pd.DataFrame(op_dict_lists[5])             

z = pd.DataFrame(z)
y = pd.DataFrame(y)

In [ ]:
roles = df.pos1.unique()
# roles

In [ ]:
roleNames = pd.DataFrame({'pos1': roles, 'num': range(len(roles))})
roleNames.columns = ['pos1', 'role']
# roleNames

In [ ]:
playerRoles = playerRoles.merge(roleNames, on = 'pos1', how = 'left')
playerRoles.columns = ['_player', 'pos1', 'role']
# playerRoles

##### Get counts of types of players (based on positions/roles) in the processed (train) data set
Also save as an Excel file for LaTeX conversion

In [ ]:
player_roles_table = playerRoles.merge(df[['name2', 'pos1', 'country', 'team']], left_on=['_player', 'pos1'], right_on=['name2', 'pos1'])
player_roles_table = player_roles_table.drop_duplicates(['name2', 'pos1'])

# counts of countries and clubs
print(len(player_roles_table['country'].unique()))
print(len(player_roles_table['team'].unique()))

player_roles_table = player_roles_table.groupby(['pos1']).agg('count')
player_roles_table = player_roles_table.reset_index()
player_roles_table = player_roles_table[['pos1', 'role']]
player_roles_table.columns = ['role', 'count']
player_roles_table = player_roles_table.sort_values(['role'])
player_roles_table.to_excel('player_roles_table.xlsx')
player_roles_table

###### Create separate "z" and "y" data for each PL club considered
- so that each team's optimization in CPLEX can be run separately, using the corresponding "z" and "y" data

In [ ]:
### for z
# first, get the list of PL clubs that are considered
# data source: www.transfermarkt.com (transfer balances)
pl_clubs_budget = pd.read_excel(os.path.join(data_path, 'PL_clubs_transfer_balances.xlsx'), sheet_name='transfer_budgets_processed')
pl_clubs_budget = pl_clubs_budget.iloc[:len(pl_clubs_budget)-1,]
# note that sofifa team names do not have " FC" suffix, hence remove " FC"
pl_clubs_budget['clubs'] = pl_clubs_budget['clubs'].str.replace(" FC", "")
pl_clubs = pl_clubs_budget[['clubs']]
# note that sofifa team names do not have " FC" suffix, hence remove " FC"
pl_clubs['clubs'] = pl_clubs['clubs'].str.replace(" FC", "")

In [ ]:
# loop for each club and create a new column for each club
pl_clubs_list = pl_clubs['clubs'].tolist()
for club in pl_clubs_list :
    z[club] = 0
for club in pl_clubs_list :
    z[club] = (z['team_0'] == club).astype(int)

In [ ]:
### for y
# For each of the 12 clubs considered, based on our subjective knowledge, we propose a list of 5 players who are "marquees." 
# These lists are based on the players that have been retained during 2015-16, 2017-18 seasons.
# It is assumed that these players won't be available for transfers, during any of the transfer windows considered.
n_marquees =  5 # number of marquee players for each club
marquee_cols = ['marquee_'+str(s) for s in range(n_marquees)]
marquee_cols
marquees = pl_clubs.copy()
for col in marquee_cols:
    marquees[col] = ''
marquees = marquees.transpose()
marquees.columns = marquees.iloc[0, :]
marquees = marquees.iloc[1:, :]

marquees['Manchester City'] = ['S. Agüero', 'K. De Bruyne', 'R. Sterling', 'David Silva', 'Fernandinho']
marquees['Manchester United'] = ['De Gea', 'A. Valencia', 'C. Smalling', 'J. Lingard', 'Ander Herrera']
marquees['Tottenham Hotspur'] = ['J. Vertonghen', 'H. Lloris', 'H. Kane', 'K. Trippier', 'D. Alli']
marquees['Liverpool'] = ['D. Sturridge', 'S. Mignolet', 'D. Lovren', 'Roberto Firmino', 'J. Gomez']
marquees['Chelsea'] = ['Azpilicueta', 'E. Hazard', 'G. Cahill', 'Willian', 'R. Loftus-Cheek']
marquees['Arsenal'] = ['A. Ramsey', 'L. Koscielny', 'M. Özil', 'Héctor Bellerín', 'P. Čech']
marquees['Everton'] = ['L. Baines', 'P. Jagielka', 'S. Naismith',  'M. Gueye', 'O. Niasse']
marquees['West Ham United'] = ['M. Noble', 'A. Cresswell', 'C. Kouyaté', 'M. Lanzini', 'M. Antonio']
marquees['Southampton'] = ['J. Ward-Prowse', 'M. Yoshida', 'R. Bertrand', 'Oriol Romeu', 'Cédric']
marquees['Swansea City'] = ['W. Routledge', 'K. Naughton', 'K. Nordfeldt', 'A. Ayew', 'L. Fer']
marquees['Stoke City'] = ['R. Shawcross', 'P. Crouch', 'C. Adam', 'J. Butland', 'M. Diouf']
marquees['West Bromwich Albion'] = ['B. Myhill', 'C. Dawson', 'C. Brunt', 'J. Morrison', 'C. Yacob']
# marquees

In [ ]:
# loop for each club and create a new column for each club
for club in pl_clubs_list :
    y[club] = y['_player'].isin(marquees[club]).astype(int)
y

In [ ]:
country_cols
for col in country_cols[:n_w]:
    hg[col] = (hg[col]=='England').astype(int)

In [ ]:
# assign some parameters that are assumed to be same for all clubs.
scalars = []
scalars.append({'param': 'pp', 'value': counter_p})
scalars.append({'param': 'tt', 'value': n_w-1}) # number of transfer windows-1 (includes the two dummy ones: initial and final)
scalars.append({'param': 'rr', 'value': len(roles)+1})
scalars.append({'param': 'nMin', 'value': 25})
scalars.append({'param': 'nMax', 'value': 40})
scalars.append({'param': 'nHGMin', 'value': 8})
scalars = pd.DataFrame(scalars)

In [ ]:
w = []
for i in range(n_w):
    if i == 0 or i == (n_w-1): # dummy transfer windows (initial and final)
        w.append({'w': 0})
    else:
        w.append({'w': 1/(n_w-2)}) # assign equal weights to all non-dummy transfer windows
w = pd.DataFrame(w)

In [ ]:
# assign buying, selling, loaning-in, and loaning-out fees as a function of market value "v"
fB = v.copy()
fB.iloc[:, 1:] = fB.iloc[:, 1:]*1.2
temp = ['_player']
temp.extend(['fB_'+str(n) for n in range(n_w)])
fB.columns = temp
fS = v.copy()
fS.iloc[:, 1:] = fS.iloc[:, 1:]*1.2
temp = ['_player']
temp.extend(['fS_'+str(n) for n in range(n_w)])
fS.columns = temp
fLI = v.copy()
fLI.iloc[:, 1:] = fLI.iloc[:, 1:]*0.12
temp = ['_player']
temp.extend(['fLI_'+str(n) for n in range(n_w)])
fLI.columns = temp
fLO = v.copy()
fLO.iloc[:, 1:] = fLO.iloc[:, 1:]*0.12
temp = ['_player']
temp.extend(['fLO_'+str(n) for n in range(n_w)])
fLO.columns = temp

In [ ]:
# assign nrMin and nrMax values based on subjective knowledge (assumed to be same for all clubs)
nrMin = {'CB':2, 
          'LB':1, 
          'ST':1, 
          'CM':1, 
          'CDM':1, 
          'RB':1, 
          'RM':1, 
          'CAM':1, 
          'LM':1, 
          'LW':1, 
          'GK':2, 
          'CF':1, 
          'RW':1, 
          'LWB':1, 
          'RWB':1}
nrMin = pd.DataFrame.from_dict(nrMin, orient='index')
nrMin = nrMin.reset_index()
nrMin.columns = ['role', 'nrMin']

nrMax = {'CB':4, 
          'LB':3, 
          'ST':4, 
          'CM':4, 
          'CDM':3, 
          'RB':3, 
          'RM':3, 
          'CAM':4, 
          'LM':3, 
          'LW':3, 
          'GK':3, 
          'CF':3, 
          'RW':3, 
          'LWB':3, 
          'RWB':3}
nrMax = pd.DataFrame.from_dict(nrMax, orient='index')
nrMax = nrMax.reset_index()
nrMax.columns = ['role', 'nrMax']

In [ ]:
# assign nOutMax and nInMax values based on subjective knowledge (assumed to be same for all clubs)
nOutMax = []
for i in range(n_w):
    if i == 0:
        nOutMax.append({'nOutMax': 0})
    else:
        nOutMax.append({'nOutMax': 10})
nOutMax = pd.DataFrame(nOutMax)

nInMax = []
for i in range(n_w):
    if i == 0:
        nInMax.append({'nInMax': 0})
    else:
        nInMax.append({'nInMax': 10})
nInMax = pd.DataFrame(nInMax)

In [ ]:
# load transfer expenses for all clubs
# data source: www.transfermarkt.com (transfer balances)
G = []
for i in range(n_w):
    if i == 0:
        G.append({'G': 0})
    else:
        G.append({'G': 150})
G = pd.DataFrame(G)

In [ ]:
# select transfer windows of interest 
# include "2015-16 winter" as the initial dummy transfer window, include "net" to represent a column for the final dummy transfer window
G = pl_clubs_budget.loc[:, ['clubs', '2015-16 winter', '2016-17 summer', '2016-17 winter', '2017-18 summer', '2017-18 winter', 'net']]
G.rename(columns={'net':'2018-19 summer'}, inplace=True)

# Here, negative values represent positive expenditure, and positive values represent negative expenditure (profit)
# Negative expenditure is replaced with zero (as profit-making from transfers may not be an objective) 
G = G.transpose()
G.columns = G.loc['clubs', :]
G = G.iloc[1:, :]
G.iloc[0, :] = 0 # set initial dummy window budget to 0
# create artificial values for final dummy window (same as previous similar window (summer/winter))
G.iloc[len(G)-1, :] = G.iloc[len(G)-3, :] 
# replace profits with 0
G[G>0] = 0 
# show negative values as positive expenditures
G[G<0] = G[G<0]*(-1)
G

In [ ]:
# save results to Excel file
writer = pd.ExcelWriter('output_Jan2019.xlsx')
scalars.to_excel(writer, 'scalars')
roleNames.to_excel(writer, 'roleNames')
playerRoles.to_excel(writer, 'playerRoles')
age.to_excel(writer, 'age')
rating.to_excel(writer, 'rating')
potential.to_excel(writer, 'potential')
hg.to_excel(writer, 'hg')
y.to_excel(writer, 'y')
z.to_excel(writer, 'z')
w.to_excel(writer, 'w')
v.to_excel(writer, 'v')
fB.to_excel(writer, 'fB')
fS.to_excel(writer, 'fS')
fLI.to_excel(writer, 'fLI')
fLO.to_excel(writer, 'fLO')
s.to_excel(writer, 's')
nrMin.to_excel(writer, 'nrMin')
nrMax.to_excel(writer, 'nrMax')
nOutMax.to_excel(writer, 'nOutMax')
nInMax.to_excel(writer, 'nInMax')
G.to_excel(writer, 'G')
writer.save()